# Evaluating Performace of Registration Methods

In [1]:
import numpy as np
import os

import planeslam.io as io
from planeslam.geometry.util import quat_to_rot_mat

Read in airsim LiDAR and pose data

In [2]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [3]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

Compute the ground truth rotations and transformations

In [4]:
# Compute the ground truth rotations
gt_R = []
for drone_orientation in drone_orientations[1:]:
    gt_R.append(quat_to_rot_mat(drone_orientation))

# Compute the groud truth translations
gt_t = drone_positions[1:] - drone_positions[0]

Evaluate the Perfomance of the Registration Methods

In [5]:
def get_R_t(filename):
    transformations = np.load(os.path.join(os.getcwd(),'registration', filename))
    R = transformations[:,:3,:3]
    t = transformations[:,:3,3]
    return R, t

p2p_R, p2p_t = get_R_t('p2p_abs_traj_transformations.npy')
p2l_R, p2l_t = get_R_t('p2l_abs_traj_transformations.npy')

In [6]:
def get_t_error(target_t, pred_t):
    return np.linalg.norm(pred_t - target_t)

def get_R_error(target_R, pred_R):
    return np.linalg.norm(pred_R @ np.linalg.inv(target_R) - np.eye(3))

p2p_t_error = get_t_error(gt_t, p2p_t)
p2p_R_error = get_R_error(gt_R, p2p_R)
print(p2p_t_error)
print(p2p_R_error)

p2l_t_error = get_t_error(gt_t, p2l_t)
p2l_R_error = get_R_error(gt_R, p2l_R)
print(p2l_t_error)
print(p2l_R_error)

148.27796578003853
0.9983505135018088
83.16566965645953
0.04508361224573657
